In [159]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [160]:
def return_calculator(df):
    period_return = 1
    li = list(df)
    for day, _ in df.iteritems():
        idx = df.index.get_loc(day)
        elem = li[idx]
        if (math.isnan(elem)):
            elem = outlier_handler(elem)
            if elem == False:
                continue
        period_return = period_return * (1+elem)
    period_return = period_return - 1
    return period_return

def outlier_handler(ret):
    if (math.isnan(ret)):
        return False

In [161]:
desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
bourse_view = os.path.join(desktop, "sector_slc", "bourse_view")
market_data_processed_path = os.path.join(bourse_view, "market_data_processed")
sectors_data_processed_path = os.path.join(bourse_view, "sectors_data_processed")
list_of_sector_processed_files = os.listdir(sectors_data_processed_path)
list_of_sector_processed_files = list_of_sector_processed_files[1:]
list_of_market_processed_files = os.listdir(market_data_processed_path)
list_of_market_processed_files = list_of_market_processed_files[1:]

In [162]:

dataset_df = pd.DataFrame()
for file_name in list_of_sector_processed_files:
    data = pd.read_excel(sectors_data_processed_path+ "\\" + file_name)
    data = data.fillna(method='ffill')

    first_year_quarter = data.quarter.iloc[0]
    first_year = data.year.iloc[0]
    last_year_quarter = data.quarter.iloc[-1]
    last_year = data.year.iloc[-1]

    empty_li_quarters = []
    empty_li_year = []

    for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_quarters += [i for i in range(first_year_quarter,5)]
            empty_li_year += [year] * (5-first_year_quarter)

        elif year == last_year:
            empty_li_quarters += [i for i in range(1,last_year_quarter+1)]
            empty_li_year += [year] * (last_year_quarter)

        else:
            empty_li_quarters += [i for i in range(1,5)]
            empty_li_year += [year] * 4

    quarterly_data = pd.DataFrame(columns=["year","quarter","quarterly_return","quarterly_trade_value","P/E-ttm"])
    quarterly_data["year"] = empty_li_year
    quarterly_data["quarter"] = empty_li_quarters

    for year in range(first_year, last_year + 1):

        if year == first_year:
            for quarter in range(first_year_quarter, 5):
                idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
                ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                quarterly_data["quarterly_return"][idx] = ret
                quarterly_data["quarterly_trade_value"][idx] = trade_value
                quarterly_data["P/E-ttm"][idx] = pe

        elif year == last_year:
            for quarter in range(1, last_year_quarter + 1):
                idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
                ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                quarterly_data["quarterly_return"][idx] = ret
                quarterly_data["quarterly_trade_value"][idx] = trade_value
                quarterly_data["P/E-ttm"][idx] = pe
        else:
            for quarter in range(1, 5):
                idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
                ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                quarterly_data["quarterly_return"][idx] = ret
                quarterly_data["quarterly_trade_value"][idx] = trade_value
                quarterly_data["P/E-ttm"][idx] = pe

    quarterly_data["P/E-ttm_1_lag"] = quarterly_data["P/E-ttm"].shift(1)
    quarterly_data["P/E-ttm_2_lag"] = quarterly_data["P/E-ttm"].shift(2)
    quarterly_data["quarterly_return_1_lag"] = quarterly_data["quarterly_return"].shift(1)
    quarterly_data["quarterly_return_2_lag"] = quarterly_data["quarterly_return"].shift(2)
    quarterly_data = quarterly_data.dropna().reset_index().drop(["index"],axis=1)

    data = pd.read_csv(market_data_processed_path+ "\\" + list_of_market_processed_files[0])
    data = data.drop(["Unnamed: 0"],axis=1)
    data = data.fillna(method='ffill')
    empty_li_quarters = []
    empty_li_year = []

    for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_quarters += [i for i in range(first_year_quarter,5)]
            empty_li_year += [year] * (5-first_year_quarter)

        elif year == last_year:
            empty_li_quarters += [i for i in range(1,last_year_quarter+1)]
            empty_li_year += [year] * (last_year_quarter)

        else:
            empty_li_quarters += [i for i in range(1,5)]
            empty_li_year += [year] * 4

    market_quarterly_data = pd.DataFrame(columns=["year","quarter","index_quarterly_return","usd_quarterly_return","quarterly_trade_value","market_pe"])
    market_quarterly_data["year"] = empty_li_year
    market_quarterly_data["quarter"] = empty_li_quarters

    for year in range(first_year, last_year + 1):

        if year == first_year:
            for quarter in range(first_year_quarter, 5):
                idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
                index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
                usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                market_quarterly_data["index_quarterly_return"][idx] = index_ret
                market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
                market_quarterly_data["quarterly_trade_value"][idx] = trade_value
                market_quarterly_data["market_pe"][idx] = pe

        elif year == last_year:
            for quarter in range(1, last_year_quarter + 1):
                idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
                index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
                usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                market_quarterly_data["index_quarterly_return"][idx] = index_ret
                market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
                market_quarterly_data["quarterly_trade_value"][idx] = trade_value
                market_quarterly_data["market_pe"][idx] = pe
        else:
            for quarter in range(1, 5):
                idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
                index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
                usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                market_quarterly_data["index_quarterly_return"][idx] = index_ret
                market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
                market_quarterly_data["quarterly_trade_value"][idx] = trade_value
                market_quarterly_data["market_pe"][idx] = pe

    market_quarterly_data["index_quarterly_return_1_lag"] = market_quarterly_data["index_quarterly_return"].shift(1)
    market_quarterly_data["index_quarterly_return_2_lag"] = market_quarterly_data["index_quarterly_return"].shift(2)
    market_quarterly_data["usd_quarterly_return_1_lag"] = market_quarterly_data["usd_quarterly_return"].shift(1)
    market_quarterly_data["usd_quarterly_return_2_lag"] = market_quarterly_data["usd_quarterly_return"].shift(2)
    market_quarterly_data["market_pe_1_lag"] = market_quarterly_data["market_pe"].shift(1)
    market_quarterly_data["market_pe_2_lag"] = market_quarterly_data["market_pe"].shift(2)
    market_quarterly_data = market_quarterly_data.dropna().reset_index().drop(["index"],axis=1)

    quarterly_data["quarterly_usd_return"] = market_quarterly_data["usd_quarterly_return"]
    quarterly_data["quarterly_usd_return_1_lag"] = market_quarterly_data["usd_quarterly_return_1_lag"]
    quarterly_data["quarterly_usd_return_2_lag"] = market_quarterly_data["usd_quarterly_return_2_lag"]
    quarterly_data["market_pe"] = market_quarterly_data["market_pe"]
    quarterly_data["market_pe_1_lag"] = market_quarterly_data["market_pe_1_lag"]
    quarterly_data["market_pe_2_lag"] = market_quarterly_data["market_pe_2_lag"]
    quarterly_data["relative_trade_value"] = 100 * quarterly_data["quarterly_trade_value"] / market_quarterly_data["quarterly_trade_value"]
    quarterly_data["index_quarterly_return"] = market_quarterly_data["index_quarterly_return"]
    quarterly_data["index_quarterly_return_1_lag"] = market_quarterly_data["index_quarterly_return_1_lag"]
    quarterly_data["index_quarterly_return_2_lag"] = market_quarterly_data["index_quarterly_return_2_lag"]
    quarterly_data["quarter_excess_return"] = quarterly_data["quarterly_return"] - quarterly_data["index_quarterly_return"]
    quarterly_data["sector"] = [file_name.split(".")[0]] * len(quarterly_data) 

    if len(dataset_df) == 0:
        dataset_df = quarterly_data

    else:
        dataset_df = pd.concat([dataset_df,quarterly_data])

In [163]:
dataset_df.to_csv("test.csv")